# This notebook is useful if you have the following:

## fatigue from typing in distance and bearings into traverse tools in arcpro
##  a set of distance and bearings, and a starting point 
## this notebook should work for a starting point in any format, as long as you set 
## spatial reference in the feature set within the database you are pointing to 

# outcome of the notebook: 
## a set of vertices from which you can construct your polygon

## Note - there is a distance and bearing to line tool in the final cell that I can't get working - i think the table that this notebook results in should suffice for that tool, but it won't work for me - if you can get that cell to work, let me know

### contact: hannah_dean@nps.gov

## notebook describing the process for taking a set of distance and bearings from a legal description and creating a set of destination points

### https://www.exceldemy.com/calculate-coordinates-from-bearing-and-distance-excel/

## Step 1 - 
### use the Scrape Legal Description for Text notebook in order to scrape out the legal descriptions - for this notebook, the direction and bearing and distance data into a spreadsheet
### the excel should have the following fields: Distance, NS (North, South) Degree, Minutes, Seconds, EW (West, East), Bearings in Radians, Latitude, Departure, Northing, Easting

### Leave the first row blank except for Northing and Easting - fill in that first Row of Northing and Easting with your starting point

#### convert quadrant bearings to true bearings - https://mathsathome.com/calculating-bearings/

### formulas for true bearings: 
### Quadrant Bearing	Conversion calculation	True Bearing
#N20°E	0° + 20°	020°
#S70°E	180° – 70°	110°
#S45°W	180° + 45°	225°
#N60°W	360° – 60°	300°
#reference: https://spreadsheetplanet.com/multiple-if-statements-in-excel/
### Excel Formula 
#####  =IF(AND(EXACT(B3,"South"),EXACT(F3,"East")),(180-G3),IF(AND(EXACT(B3,"South"),EXACT(F3,"West")),(180+G3),IF(AND(EXACT(B3,"North"),EXACT(F3,"East")),(G3),IF(AND(EXACT(B3,"North"),EXACT(F3,"West")),(360-G3)))))

In [1]:
#if you'd rather scrape data and do calculations in this notebook use this cell 
# Import packages
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#point the sample to the pathway for your spreadsheet
sample = r"C:\Users\hdean\Documents\ArcGIS\Projects\PythonWorkSpace_ROW\Sample_legaldescrptions.xlsx"

# display your data
#pd.read_excel(sample)


In [2]:
#Calculate Decimal degrees (DD) field
# Reference https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.ExcelWriter.html
excel_file = pd.ExcelFile(sample) 
df = excel_file.parse('rawdata') 
#print(df)

#show top few rows of data frame
df.head()



,Distance,NS,Degree,Minutes,Seconds,EW,DD,Bearing in Radians,Latitude,Departure,Northing,Easting
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151651.948,1016284.67
1,15.01,South,19.0,26.0,24.0,East,NaN,NaN,NaN,NaN,NaN,NaN
2,273.69,South,68.0,53.0,12.0,West,NaN,NaN,NaN,NaN,NaN,NaN
3,108.39,South,45.0,44.0,1.0,West,NaN,NaN,NaN,NaN,NaN,NaN
4,403.92,South,72.0,8.0,3.0,West,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Add column called truebearing and then calculate that field
df['truebearing']= np.nan

#calculate the Decimal Degrees, 'DD' Column
#reference https://pythoninoffice.com/create-calculated-columns-in-a-dataframe/
df['DD'] = df['Degree']+(df['Minutes']/60)+(df['Seconds']/3600)

# convert quadrant bearings in Decimal Degrees - 'DD' - to true bearings - 
#reference: https://mathsathome.com/calculating-bearings/

#formulas for true bearings: 
#Quadrant Bearing	Conversion calculation	True Bearing
#N20°E	0° + 20°	020°
#S70°E	180° – 70°	110°
#S45°W	180° + 45°	225°
#N60°W	360° – 60°	300°
#reference: https://spreadsheetplanet.com/multiple-if-statements-in-excel/

# Excel Formula 
#  =IF(AND(EXACT(B3,"South"),EXACT(F3,"East")),(180-G3),IF(AND(EXACT(B3,"South"),EXACT(F3,"West")),(180+G3),IF(AND(EXACT(B3,"North"),EXACT(F3,"East")),(G3),IF(AND(EXACT(B3,"North"),EXACT(F3,"West")),(360-G3)))))

#REFERENCE https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
def conditions(df):
    if (df['NS']=='South') and (df['EW'] == 'East'): 
        return (180-df['DD']) 
    elif (df['NS']=='South') and (df['EW'] == 'West'): 
        return (180+df['DD'])
    elif (df['NS']=='North') and (df['EW'] == 'East'): 
        return (df['DD'])
    elif (df['NS']=='North') and (df['EW'] == 'West'): 
        return (360-df['DD'])

df['truebearing']=df.apply(conditions, axis = 1)

df.head()


,Distance,NS,Degree,Minutes,Seconds,EW,DD,Bearing in Radians,Latitude,Departure,Northing,Easting,truebearing
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151651.948,1016284.67,NaN
1,15.01,South,19.0,26.0,24.0,East,19.440000,NaN,NaN,NaN,NaN,NaN,160.560000
2,273.69,South,68.0,53.0,12.0,West,68.886667,NaN,NaN,NaN,NaN,NaN,248.886667
3,108.39,South,45.0,44.0,1.0,West,45.733611,NaN,NaN,NaN,NaN,NaN,225.733611
4,403.92,South,72.0,8.0,3.0,West,72.134167,NaN,NaN,NaN,NaN,NaN,252.134167


In [4]:
#calculate the Radians Column using numpy
#reference https://numpy.org/doc/stable/reference/generated/numpy.deg2rad.html

import numpy as np 
import math 
df['Bearing in Radians'] = np.deg2rad(df['truebearing'])

df.head()

,Distance,NS,Degree,Minutes,Seconds,EW,DD,Bearing in Radians,Latitude,Departure,Northing,Easting,truebearing
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151651.948,1016284.67,NaN
1,15.01,South,19.0,26.0,24.0,East,19.440000,2.802301,NaN,NaN,NaN,NaN,160.560000
2,273.69,South,68.0,53.0,12.0,West,68.886667,4.343892,NaN,NaN,NaN,NaN,248.886667
3,108.39,South,45.0,44.0,1.0,West,45.733611,3.939795,NaN,NaN,NaN,NaN,225.733611
4,403.92,South,72.0,8.0,3.0,West,72.134167,4.400571,NaN,NaN,NaN,NaN,252.134167


In [5]:
#numpy reference for cosine and sin functions https://numpy.org/doc/stable/reference/generated/numpy.sin.html

#calculate Latitude
# Excel Formula =A3*COS(I3)
df['Latitude'] = df['Distance']*np.cos(df['Bearing in Radians'])

#calculate Departure
# Excel Formula =A3*SIN(I3)
df['Departure'] = df['Distance']*np.sin(df['Bearing in Radians'])

df.head()



,Distance,NS,Degree,Minutes,Seconds,EW,DD,Bearing in Radians,Latitude,Departure,Northing,Easting,truebearing
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151651.948,1016284.67,NaN
1,15.01,South,19.0,26.0,24.0,East,19.440000,2.802301,-14.154288,4.995621,NaN,NaN,160.560000
2,273.69,South,68.0,53.0,12.0,West,68.886667,4.343892,-98.586944,-255.317118,NaN,NaN,248.886667
3,108.39,South,45.0,44.0,1.0,West,45.733611,3.939795,-75.655713,-77.618330,NaN,NaN,225.733611
4,403.92,South,72.0,8.0,3.0,West,72.134167,4.400571,-123.918256,-384.441975,NaN,NaN,252.134167


In [6]:
#calculate Northings and Eastings

# Excel formula =J3+L2 = ['Latitude']+[previouse 'Northing' value]
#Excel formala for Easting = ['Departure']+[previous Easting value]


#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html
#rather than referencing the previous cell as you would in excel in order
#to incrementally add Latitudes, the solution here is to use cumsum
#this will require entering the first Northing into the formula manually
#Reference - cumsum - https://stackoverflow.com/questions/41859311/cumsum-as-a-new-column-in-an-existing-pandas-dataframe
#pandas doc - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.cumsum.html


df['Northings_cum']=151651.948+df['Latitude'].cumsum()

df['Eastings_cum'] = 1016284.67+df['Departure'].cumsum()
    

df.head()
                                        
                                       

,Distance,NS,Degree,Minutes,Seconds,EW,DD,Bearing in Radians,Latitude,Departure,Northing,Easting,truebearing,Northings_cum,Eastings_cum
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151651.948,1016284.67,NaN,NaN,NaN
1,15.01,South,19.0,26.0,24.0,East,19.440000,2.802301,-14.154288,4.995621,NaN,NaN,160.560000,151637.793712,1.016290e+06
2,273.69,South,68.0,53.0,12.0,West,68.886667,4.343892,-98.586944,-255.317118,NaN,NaN,248.886667,151539.206768,1.016034e+06
3,108.39,South,45.0,44.0,1.0,West,45.733611,3.939795,-75.655713,-77.618330,NaN,NaN,225.733611,151463.551055,1.015957e+06
4,403.92,South,72.0,8.0,3.0,West,72.134167,4.400571,-123.918256,-384.441975,NaN,NaN,252.134167,151339.632799,1.015572e+06


In [7]:
df.dtypes
csvpath = r"C:\Users\hdean\Documents\ArcGIS\Projects\PythonWorkSpace_ROW\out1.csv"
fc_path = r"C:\Users\hdean\Documents\ArcGIS\Projects\PythonWorkSpace_ROW\MyProject43.gdb\LongIsland"
outFC = r"C:\Users\hdean\Documents\ArcGIS\Projects\PythonWorkSpace_ROW\MyProject43.gdb\LongIsland/points"
gdbpath = r"C:\Users\hdean\Documents\ArcGIS\Projects\PythonWorkSpace_ROW\MyProject43.gdb"

#REFERENCE - https://pro.arcgis.com/en/pro-app/latest/tool-reference/conversion/table-to-table.htm

#convert df to csv, then convert csv to table
df.to_csv(csvpath)

#arcpy.conversion.TableToTable(in_rows, out_path, out_name, {where_clause}, {field_mapping}, {config_keyword})

arcpy.conversion.TableToTable(csvpath,gdbpath,"table1")



<Result 'C:\\Users\\hdean\\Documents\\ArcGIS\\Projects\\PythonWorkSpace_ROW\\MyProject43.gdb\\table1'>

In [8]:
#use distance and bearing line tool
#REFERENCE - https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/bearing-distance-to-line.htm


intable = r"C:\Users\hdean\Documents\ArcGIS\Projects\PythonWorkSpace_ROW\MyProject43.gdb\table1"
verts = r"C:\Users\hdean\Documents\ArcGIS\Projects\PythonWorkSpace_ROW\MyProject43.gdb\verts2"

#xy table to point ref - https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/xy-table-to-point.htm
#https://pro.arcgis.com/en/pro-app/latest/arcpy/classes/spatialreference.htm

sr = arcpy.Describe(fc_path).SpatialReference
print(sr.name)

arcpy.management.XYTableToPoint(intable, verts, 'Eastings_cum','Northings_cum',"",sr.name)

NAD_1983_HARN_StatePlane_New_York_Long_Island_FIPS_3104_Feet


<Result 'C:\\Users\\hdean\\Documents\\ArcGIS\\Projects\\PythonWorkSpace_ROW\\MyProject43.gdb\\verts2'>

In [26]:
startpoint = r"C:\Users\hdean\Documents\ArcGIS\Projects\PythonWorkSpace_ROW\MyProject43.gdb\startpoint"
allpoints = r"C:\Users\hdean\Documents\ArcGIS\Projects\PythonWorkSpace_ROW\MyProject43.gdb\allpoints"

#map starting point
arcpy.management.XYTableToPoint(intable, startpoint, 'Easting','Northing',"",sr.name)

#merge starting point with all vertices
#arcpy.management.Merge(inputs, output, {field_mappings}, {add_source})

import arcpy
arcpy.Merge_management([verts, startpoint],allpoints)


<Result 'C:\\Users\\hdean\\Documents\\ArcGIS\\Projects\\PythonWorkSpace_ROW\\MyProject43.gdb\\allpoints'>

In [28]:
#map line using bearing distance to line 
#referece https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/bearing-distance-to-line.htm
outline = r"C:\Users\hdean\Documents\ArcGIS\Projects\PythonWorkSpace_ROW\MyProject43.gdb\LongIsland/line2"

#arcpy.management.BearingDistanceToLine(in_table, out_featureclass, x_field, y_field, distance_field, {distance_units}, bearing_field, {bearing_units}, {line_type}, {id_field}, {spatial_reference}, {attributes})

arcpy.management.BearingDistanceToLine(intable,outline,'Eastings_cum','Northings_cum','Distance','US_SURVEY_FEET','Bearing in Radians','RADS',"","",sr.name)

<Result 'C:\\Users\\hdean\\Documents\\ArcGIS\\Projects\\PythonWorkSpace_ROW\\MyProject43.gdb\\LongIsland\\line2'>